In [1]:
import json
from pathlib import Path

import requests
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup

In [2]:
p = Path('hse_courses')
p.mkdir(exist_ok=True)

In [3]:
base_url = 'https://www.hse.ru'
url = '/edu/courses/page1.html'
done = int(url[url.rfind("/")+1:].split('.')[0][4:]) - 1
raw = requests.get(base_url + url).text
soup = BeautifulSoup(raw, 'lxml')

In [4]:
last = soup.find('div', 'letterlist').findChildren(recursive=False)[-1]['href']
last = last[last.rfind("/")+1:]
total = int(last.split('.')[0][4:])

In [5]:
with tqdm(total=total) as t:
    raw = requests.get(base_url + url).text
    soup = BeautifulSoup(raw, 'lxml')
    done = int(url[url.rfind("/")+1:].split('.')[0][4:]) - 1
    t.update(done)
    while True:
        page_data = []
        for program in soup.find_all('div', 'b-program'):
            program = program.div
            name, year, fields = program.findChildren(recursive=False)
            data = {
                'course_name': name.text,
                'course_url': name.a['href'],
                'year': str(year.contents[1])
            }
            
            fields_data = {}
            for field in fields.findChildren(recursive=False):
                title = field.contents[0]
                k = title.text.replace(':', '').strip()
                title.decompose()
                fields_data[k] = {
                    'text': field.text.strip(),
                    'links': [a['href'] for a in field.find_all('a', href=True)]
                }
            data['fields'] = fields_data
            page_data.append(data)
            
        fname = url[url.rfind("/")+1:].split('.')[0] + '.json'
        with p.joinpath(fname).open('w', encoding='utf8') as f:
            json.dump(page_data, f, ensure_ascii=False, indent=2)
        
        t.update(1)
        next_url = soup.find('div', 'letterlist').findChildren(recursive=False)[-2]
        if next_url.text == '→':
            url = next_url['href']
            raw = requests.get(base_url + url).text
            soup = BeautifulSoup(raw, 'lxml')
        else:
            next_url = soup.find('div', 'letterlist').findChildren(recursive=False)[-1]
            if next_url.text == '→':
                url = next_url['href']
            else:
                break
        try:
            raw = requests.get(base_url + url).text
        except:
            raw = requests.get(base_url + url).text  # repeat request once if something went wrong with connecting the first time
        soup = BeautifulSoup(raw, 'lxml')

HBox(children=(IntProgress(value=0, max=412), HTML(value='')))

In [6]:
composed_data = []
for j_p in p.iterdir():
    if j_p.suffix != '.json':
        continue
    composed_data += json.loads(j_p.read_text(encoding='utf8'))

In [10]:
with open('composed_courses.json', 'w', encoding='utf8') as f:
    json.dump(composed_data, f, ensure_ascii=False, indent=2)